# Leader data analysis

In [1]:
## Import packages
library(glmnet)
library(zeallot) #enable %<-%
library(mvtnorm)
library(causl)
library(survivl)
library(npcausal)
library(dplyr)
library(haven)
library(survival)

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: rje

Loading required package: VineCopula


Attaching package: 'dplyr'


The following object is masked from 'package:rje':

    last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:

## baseline ####
bsl_vars <- c("SEX", "AGE", "RACE", "SMOKER", "DIABDUR", "BMIBL", "HBA1CBL",
              "HDL1BL", "LDL1BL", "CHOL1BL", "TRIG1BL", "CREATBL")

mh_vars <- c("MIFL","STROKEFL","STENFL", "NEPSCRFL", "KIDFL")

## longitudinal ####
lb_vars <- c("HBA1C", "EGFRCKD")
vs_vars <- "BMI"

## time to event ####
tte_vars <- c("ALDTHTM", "MACEEVTM", "MACEMITM")


# Load script that goes through both folders - credit to Jens
path = "../../../../Project/LEADER/Box/"
folder = list(start = paste0(path, "Analysis Ready Datasets/SAS_analysis/"),
              fin = paste0(path, "Analysis Ready Datasets/R_analysis/"))

load_func <- function(ds, folder){
  if (file.exists(paste0(folder$fin, ds, ".rds"))){
    out <- readRDS(paste0(folder$fin, ds, ".rds"))
  } else {
    out <- haven::read_sas(paste0(folder$start, ds, ".sas7bdat"))
    saveRDS(out, paste0(folder$fin, ds, ".rds"))
  }
  return(out)
}


In [3]:
# Load in tables 
adsl <- load_func("adsl", folder) # demographics
advs <- load_func("advs", folder) # vitals
adtte <- load_func("adtte", folder) # primary outcomes
adlb <- load_func("adlb", folder) # labs; can be slow

In [4]:
df_bsl <- adsl %>%
  filter(FASFL == "Y") %>% 
  select(USUBJID, ARM, all_of(bsl_vars), all_of(mh_vars))

df_long <- rbind(
  adlb %>% 
    filter(FASFL == "Y") %>% 
    filter(PARAMCD %in% lb_vars) %>% 
    mutate(DS = "adlb") %>% 
    select(DS, USUBJID, ABLFL, AVISIT, AVISITN, ADY,
           PARAM, PARAMCD, AVALU, AVAL, CHG, PCHG, DTYPE),
  advs %>% 
    filter(FASFL == "Y") %>% 
    filter(PARAMCD %in% vs_vars) %>% 
    mutate(DS = "advs") %>% 
    select(DS, USUBJID, ABLFL, AVISIT, AVISITN, ADY,
           PARAM, PARAMCD, AVALU, AVAL, CHG, PCHG, DTYPE)
)

death <- adtte %>% 
  filter(FASFL == "Y") %>% 
  filter(PARAMCD == "ALDTHTM") %>% 
  mutate(death = ifelse(CNSR == 1, 0, 1))

df_out <- adtte %>% 
  filter(FASFL == "Y") %>% 
  filter(PARAMCD %in% tte_vars) %>% 
  mutate(event = ifelse(CNSR == 1, 0, 1)) %>% 
  select(USUBJID, PARAM, PARAMCD, AVAL, event) %>% 
  left_join(death %>% select(USUBJID, death)) %>% 
  mutate(death = ifelse(event == 1, 0, death))

out <- list(
  df_bsl = df_bsl,
  df_long = df_long, 
  df_out = df_out
)

Joining with `by = join_by(USUBJID)`


In [5]:
df_bsl <- adsl %>%
  filter(FASFL == "Y") %>% 
  select(USUBJID, ARM, all_of(bsl_vars), all_of(mh_vars))
nrow(df_bsl)

[1] 9340

In [6]:
df_bsl

USUBJID,ARM,SEX,AGE,RACE,SMOKER,DIABDUR,BMIBL,HBA1CBL,HDL1BL,LDL1BL,CHOL1BL,TRIG1BL,CREATBL,MIFL,STROKEFL,STENFL,NEPSCRFL,KIDFL
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
EX2211-3748/5,Liraglutide,F,62,WHITE,NEVER SMOKED,3.5,32.7,9.5,1.17,2.80,4.84,1.90,55,N,N,N,N,N
EX2211-3748/9,Placebo,M,56,WHITE,CURRENT SMOKER,3.2,26.9,6.8,0.87,1.82,3.27,1.28,77,N,N,N,N,N
EX2211-3748/16,Liraglutide,M,60,WHITE,PREVIOUS SMOKER,13.9,31.5,11.9,0.85,2.42,4.17,1.97,70,Y,N,N,N,N
EX2211-3748/20,Placebo,F,59,WHITE,NEVER SMOKED,2.1,43.9,9.6,1.14,2.49,4.84,2.64,55,N,N,Y,N,N
EX2211-3748/62,Liraglutide,M,71,BLACK OR AFRICAN AMERICAN,PREVIOUS SMOKER,11.5,28.4,8.5,1.06,2.02,4.53,3.14,81,N,N,N,N,N
EX2211-3748/65,Placebo,M,56,OTHER,NEVER SMOKED,8.0,28.2,7.6,0.88,1.63,3.08,1.27,114,N,N,N,Y,N
EX2211-3748/71,Placebo,M,60,BLACK OR AFRICAN AMERICAN,CURRENT SMOKER,19.3,25.1,8.0,1.04,2.03,4.49,3.13,80,N,N,N,Y,N
EX2211-3748/73,Placebo,M,67,WHITE,PREVIOUS SMOKER,11.2,35.1,8.8,0.98,1.74,3.76,2.25,76,N,N,N,N,N
EX2211-3748/89,Placebo,M,64,ASIAN,NEVER SMOKED,17.1,23.3,7.0,0.96,1.89,3.60,1.65,116,N,N,N,N,N


In [7]:
# Load required package
library(dplyr)

# Step 1. Convert ARM into treatment indicator:
df_bsl <- df_bsl %>%
  mutate(ARM = ifelse(ARM == "Liraglutide", 1,
                      ifelse(ARM == "Placebo", 0, NA)))

# Step 2. Process all covariate columns (all except USUBJID and ARM):
# Create an empty list to hold processed columns
cov_list <- list()
# Get the names of the columns to process
cov_names <- setdiff(names(df_bsl), c("USUBJID", "ARM"))

# Loop over each covariate column
for(col in cov_names) {
  column_data <- df_bsl[[col]]
  
  # If the column is a character or factor type:
  if(is.character(column_data) || is.factor(column_data)) {
    # Check if the column is binary with "N" and "Y"
    if(all(unique(as.character(column_data)) %in% c("N", "Y"))) {
      cov_list[[col]] <- ifelse(column_data == "Y", 1, 0)
    } else {
      # Otherwise, create dummy variables for a multi-level categorical variable.
      # model.matrix creates one column per level (no intercept).
      dummies <- model.matrix(~ column_data - 1)
      # Drop the last dummy column if more than one dummy variable is created.
      if(ncol(dummies) > 1){
        dummies <- dummies[, -ncol(dummies), drop = FALSE]
      }
      # Optionally, rename dummy columns to include original column name as prefix
      dummy_names <- paste(col, sub("column_data", "", colnames(dummies)), sep = "_")
      colnames(dummies) <- dummy_names
      # Add each dummy column into our list
      for(j in seq_along(dummy_names)) {
        cov_list[[ dummy_names[j] ]] <- dummies[, j]
      }
    }
  } else {
    # For numeric columns, just include them unchanged
    cov_list[[col]] <- column_data
  }
}

# Combine the processed covariate columns into one data frame:
cov_df <- as.data.frame(cov_list)

# Step 3. Reorder the covariate columns so that binary ones come first.
# Here we define a binary column as one that (ignoring NAs) only takes values 0 and 1.
is_binary <- sapply(cov_df, function(x) {
  vals <- unique(x[!is.na(x)])
  length(vals) == 2 && all(sort(vals) == c(0, 1))
})
# Order: binary columns first, then the rest
cov_df <- cov_df[, c(names(cov_df)[is_binary], names(cov_df)[!is_binary])]

# Step 4. Rename all covariate columns as X_1, X_2, …, X_p
new_names <- paste0("X_", seq_along(cov_df))
colnames(cov_df) <- new_names

# Combine USUBJID, ARM with the newly processed covariates
final_df <- cbind(df_bsl[, c("USUBJID", "ARM")], cov_df)

# final_df now has USUBJID, ARM and the sorted & renamed covariates.


In [8]:
final_df

,USUBJID,ARM,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,EX2211-3748/5,1,1,0,0,0,0,0,0,1,...,0,62,3.5,32.7,9.5,1.17,2.80,4.84,1.90,55
2,EX2211-3748/9,0,0,0,0,0,0,0,1,0,...,0,56,3.2,26.9,6.8,0.87,1.82,3.27,1.28,77
3,EX2211-3748/16,1,0,0,0,0,0,0,0,0,...,0,60,13.9,31.5,11.9,0.85,2.42,4.17,1.97,70
4,EX2211-3748/20,0,1,0,0,0,0,0,0,1,...,0,59,2.1,43.9,9.6,1.14,2.49,4.84,2.64,55
5,EX2211-3748/62,1,0,0,0,1,0,0,0,0,...,0,71,11.5,28.4,8.5,1.06,2.02,4.53,3.14,81
6,EX2211-3748/65,0,0,0,0,0,0,1,0,1,...,0,56,8.0,28.2,7.6,0.88,1.63,3.08,1.27,114
7,EX2211-3748/71,0,0,0,0,1,0,0,1,0,...,0,60,19.3,25.1,8.0,1.04,2.03,4.49,3.13,80
8,EX2211-3748/73,0,0,0,0,0,0,0,0,0,...,0,67,11.2,35.1,8.8,0.98,1.74,3.76,2.25,76
9,EX2211-3748/89,0,0,0,1,0,0,0,0,1,...,0,64,17.1,23.3,7.0,0.96,1.89,3.60,1.65,116


In [9]:
df_long

DS,USUBJID,ABLFL,AVISIT,AVISITN,ADY,PARAM,PARAMCD,AVALU,AVAL,CHG,PCHG,DTYPE
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
adlb,EX2211-3748/5,,"VISIT 1, V10",10,-35,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,97.7,NA,NA,
adlb,EX2211-3748/5,Y,"VISIT 3 (DAY 0), V30",30,2,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,96.6,0.0,0.00,
adlb,EX2211-3748/5,,"VISIT 6 (MONTH 6), V60",60,181,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,94.9,-1.7,-1.76,
adlb,EX2211-3748/5,,"VISIT 7 (MONTH 12), V70",70,371,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,94.4,-2.2,-2.28,
adlb,EX2211-3748/5,,"VISIT 9 (MONTH 24), V90",90,727,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,93.8,-2.8,-2.90,
adlb,EX2211-3748/5,,"VISIT 11 (MONTH 36), V110",110,1098,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,94.4,-2.2,-2.28,
adlb,EX2211-3748/5,,"VISIT 13 (MONTH 48), V130",130,1098,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,94.4,-2.2,-2.28,LOCF
adlb,EX2211-3748/5,,"VISIT 15 (MONTH 60), V150",150,1358,EGFR using CKD-EPI Method (ml/min/1.73m2),EGFRCKD,ml/min/1.73m2,92.8,-3.8,-3.93,
adlb,EX2211-3748/5,,"VISIT 1, V10",10,-35,Hemoglobin A1C (%),HBA1C,%,13.3,NA,NA,


In [10]:
max(df_out$AVAL)

[1] 62.32444

In [16]:
library(dplyr)
library(tidyr)
library(purrr)
# Determine maximum follow-up time and number of intervals (each of 6 months)
max_time <- max(df_out$AVAL)
num_intervals <- ceiling(max_time / 6)

# Separate the outcomes:
# MACE outcome (use PARAMCD "MACEEVTM")
mace <- df_out %>% 
  filter(PARAMCD == "MACEEVTM") %>% 
  select(USUBJID, time = AVAL, event)

# Death outcome (use PARAMCD "ALDTHTM")
death <- df_out %>% 
  filter(PARAMCD == "ALDTHTM") %>% 
  select(USUBJID, time = AVAL, event = death)

# Non-fatal MI outcome (use PARAMCD "MACEMITM")
mi <- df_out %>% 
  filter(PARAMCD == "MACEMITM") %>% 
  select(USUBJID, time = AVAL, event)

# Create a wide table that has one row per subject and merge outcomes
surv_table <- df_out %>% 
  distinct(USUBJID) %>% 
  left_join(mace, by = "USUBJID") %>% 
  rename(time_mace = time, event_mace = event) %>%
  left_join(death, by = "USUBJID") %>% 
  rename(time_death = time, event_death = event) %>%
  left_join(mi, by = "USUBJID") %>% 
  rename(time_mi = time, event_mi = event)

# Helper function to create the interval vector:
# - time: the time to event (in months)
# - event: indicator (1 = event occurred, 0 = censored/no event)
# - num_intervals: total number of intervals
# - interval_length: length of each interval (4 months here)
create_interval_vector <- function(time, event, num_intervals, interval_length = 6) {
  res <- rep(0, num_intervals)
  if(event == 1) {
    # Determine which interval the event falls into
    event_interval <- ceiling(time / interval_length)
    if(event_interval > num_intervals) event_interval <- num_intervals
    res[event_interval] <- 1
    # Set subsequent intervals to NA once the event occurs
    if(event_interval < num_intervals) {
      res[(event_interval + 1):num_intervals] <- NA
    }
  }
  return(res)
}

# Apply the function to each outcome
surv_table <- surv_table %>%
  mutate(
    Y = map2(time_mace, event_mace, ~create_interval_vector(.x, .y, num_intervals)),
    D = map2(time_death, event_death, ~create_interval_vector(.x, .y, num_intervals)),
    I = map2(time_mi, event_mi, ~create_interval_vector(.x, .y, num_intervals))
  )

# Expand the list columns into separate columns for each interval:
for (i in 1:num_intervals) {
  surv_table[[paste0("Y", i)]] <- map_dbl(surv_table$Y, ~.x[i])
  surv_table[[paste0("D", i)]] <- map_dbl(surv_table$D, ~.x[i])
  surv_table[[paste0("I", i)]] <- map_dbl(surv_table$I, ~.x[i])
}

# Select the final columns: one row per subject and columns for each interval and outcome
final_table <- surv_table %>% 
  select(USUBJID, starts_with("Y"), starts_with("D"), starts_with("I"))

# View the final table
print(final_table)

# A tibble: 9,340 x 37
   USUBJID     Y        Y1    Y2    Y3    Y4    Y5    Y6    Y7    Y8    Y9   Y10
   <chr>       <lis> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
 2 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
 3 EX2211-374~ <dbl>     0     0     0     0     1    NA    NA    NA    NA    NA
 4 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
 5 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
 6 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
 7 EX2211-374~ <dbl>     1    NA    NA    NA    NA    NA    NA    NA    NA    NA
 8 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
 9 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
10 EX2211-374~ <dbl>     0     0     0     0     0     0     0     0     0     0
# i 9

In [21]:
final_table[3,'Y']

Y
<list>
0
0
0
0
1
NA
NA
NA
NA


In [15]:
mace

USUBJID,time,event
<chr>,<dbl>,<dbl>
EX2211-3748/5,48.32854,0
EX2211-3748/9,43.53183,0
EX2211-3748/16,28.25462,1
EX2211-3748/20,44.84600,0
EX2211-3748/62,48.39425,0
EX2211-3748/65,55.91786,0
EX2211-3748/71,4.36961,1
EX2211-3748/73,52.13963,0
EX2211-3748/89,43.40041,0


In [26]:
final_table[1,'Y']

Y
<list>
0
0
0
0
0
0
0
0
0


In [14]:
library(dplyr)
library(tidyr)
library(purrr)

#--- 1) Create separate wide tables for each parameter ------------------
df_long %>%
  group_by(USUBJID, AVISIT, AVISITN, PARAMCD) %>%
  summarise(n = n(), .groups = "drop") %>%
  filter(n > 1)

# EGFRCKD table
df_egfr <- df_long %>% 
  filter(PARAMCD == "EGFRCKD") %>%
  select(USUBJID, AVISIT, AVISITN, AVAL) %>%
  pivot_wider(names_from = AVISITN, values_from = AVAL, values_fill = NA, values_fn = first)

# HBA1C table
df_hba1c <- df_long %>%
  filter(PARAMCD == "HBA1C") %>%
  select(USUBJID, AVISIT, AVISITN, AVAL) %>%
  pivot_wider(names_from = AVISITN, values_from = AVAL, values_fill = NA, values_fn = first)

# BMI table (if BMI records exist, otherwise this will be empty)
df_bmi <- df_long %>%
  filter(PARAMCD == "BMI") %>%
  select(USUBJID, AVISIT, AVISITN, AVAL) %>%
  pivot_wider(names_from = AVISITN, values_from = AVAL, 
            values_fill = NA, values_fn = first)

#--- 2) Create the combined table with list cells --------------------------

# (a) Build a complete grid of visits per subject.
visits <- df_long %>%
  select(USUBJID, AVISIT, AVISITN) %>%
  distinct()

# (b) Pivot the long data so that each row (per USUBJID & AVISIT) gets a column for each PARAMCD.
df_combined_long <- df_long %>%
  select(USUBJID, AVISIT, AVISITN, PARAMCD, AVAL) %>%
  pivot_wider(names_from = PARAMCD, values_from = AVAL, values_fill = NA, values_fn=first)

# (c) If BMI is missing entirely, add it manually with NA.
if (!"BMI" %in% names(df_combined_long)) {
  df_combined_long <- df_combined_long %>% mutate(BMI = NA)
}

# (d) Join with the complete grid to ensure every visit is present.
df_combined_long <- visits %>%
  left_join(df_combined_long, by = c("USUBJID", "AVISIT", "AVISITN"))

# (e) For each row, create a combined list (with names) of the three parameters.
df_list_long <- df_combined_long %>%
  mutate(combined = pmap(list(EGFRCKD, HBA1C, BMI),
                         ~ list(EGFRCKD = ..1, HBA1C = ..2, BMI = ..3))) %>%
  select(USUBJID, AVISIT, AVISITN, combined)

# (f) Pivot wider so each USUBJID is one row and each AVISITN is a column containing the list.
df_list <- df_list_long %>%
  select(USUBJID, AVISITN, combined) %>%
  pivot_wider(names_from = AVISITN, values_from = combined)

# Now you have four data frames:
#   • df_egfr  - wide table for EGFRCKD (one column per visit)
#   • df_hba1c - wide table for HBA1C (one column per visit)
#   • df_bmi   - wide table for BMI (one column per visit; if BMI is missing, these columns will be NA)
#   • df_list  - combined wide table with list cells; each cell is a list [EGFRCKD, HBA1C, BMI]


USUBJID,AVISIT,AVISITN,PARAMCD,n
<chr>,<chr>,<dbl>,<chr>,<int>
EX2211-3748/10023,"VISIT 15 (MONTH 60), V150",150,EGFRCKD,2
EX2211-3748/10023,"VISIT 6 (MONTH 6), V60",60,HBA1C,2
EX2211-3748/10034,"VISIT 3 (DAY 0), V30",30,HBA1C,2
EX2211-3748/10034,"VISIT 5 (MONTH 3), V50",50,HBA1C,2
EX2211-3748/10085,"VISIT 5 (MONTH 3), V50",50,HBA1C,2
EX2211-3748/10087,"VISIT 3 (DAY 0), V30",30,EGFRCKD,2
EX2211-3748/10087,"VISIT 3 (DAY 0), V30",30,HBA1C,2
EX2211-3748/10203,"VISIT 8 (MONTH 18), V80",80,HBA1C,2
EX2211-3748/10203,"VISIT 9 (MONTH 24), V90",90,EGFRCKD,2


Warning message:
"Values from `combined` are not uniquely identified; output will contain
list-cols.
* Use `values_fn = list` to suppress this warning.
* Use `values_fn = {summary_fun}` to summarise duplicates.
* Use the following dplyr code to identify duplicates.
  {data} |>
  dplyr::summarise(n = dplyr::n(), .by = c(USUBJID, AVISITN)) |>
  dplyr::filter(n > 1L)"


In [15]:
library(dplyr)
library(tidyr)
library(purrr)
library(stringr)

# First, add a new column 'month' that extracts the month number.
# If "DAY 0" appears in AVISIT, mark month as "0"; otherwise, extract the digits after "MONTH ".
df_long <- df_long %>%
  mutate(month = if_else(str_detect(AVISIT, "DAY\\s*0"),
                         "0",
                         str_extract(AVISIT, "(?<=MONTH\\s)\\d+")))

# Optionally, filter to keep only rows that have a month value (if desired)
df_long_month <- df_long %>% 
  filter(!is.na(month))

#--- 1) Create separate wide tables for each parameter ------------------

# EGFRCKD table
df_egfr <- df_long_month %>%
  filter(PARAMCD == "EGFRCKD") %>%
  select(USUBJID, month, AVAL) %>%
  group_by(USUBJID, month) %>% 
  summarise(AVAL = first(AVAL), .groups = "drop") %>%
  pivot_wider(id_cols = USUBJID, 
              names_from = month, 
              values_from = AVAL)

# HBA1C table
df_hba1c <- df_long_month %>%
  filter(PARAMCD == "HBA1C") %>%
  select(USUBJID, month, AVAL) %>%
  group_by(USUBJID, month) %>% 
  summarise(AVAL = first(AVAL), .groups = "drop") %>%
  pivot_wider(id_cols = USUBJID, 
              names_from = month, 
              values_from = AVAL)

# BMI table
df_bmi <- df_long_month %>%
  filter(PARAMCD == "BMI") %>%
  select(USUBJID, month, AVAL) %>%
  group_by(USUBJID, month) %>% 
  summarise(AVAL = first(AVAL), .groups = "drop") %>%
  pivot_wider(id_cols = USUBJID, 
              names_from = month, 
              values_from = AVAL)

#--- 2) Create the combined table with list-cells -------------------------------

# Summarize so each subject-month (visit) gets one row per parameter.
df_summary <- df_long_month %>%
  select(USUBJID, month, PARAMCD, AVAL) %>%
  group_by(USUBJID, month, PARAMCD) %>%
  summarise(AVAL = first(AVAL), .groups = "drop") %>%
  pivot_wider(names_from = PARAMCD, values_from = AVAL)

# Ensure that all three parameters exist as columns.
if (!"BMI" %in% colnames(df_summary)) {
  df_summary <- df_summary %>% mutate(BMI = NA)
}
if (!"EGFRCKD" %in% colnames(df_summary)) {
  df_summary <- df_summary %>% mutate(EGFRCKD = NA)
}
if (!"HBA1C" %in% colnames(df_summary)) {
  df_summary <- df_summary %>% mutate(HBA1C = NA)
}

# Combine the three values into a list for each subject-month.
df_summary <- df_summary %>%
  mutate(combined = pmap(list(EGFRCKD, HBA1C, BMI),
                         ~ list(EGFRCKD = ..1, HBA1C = ..2, BMI = ..3))) %>%
  select(USUBJID, month, combined)

# Pivot wider so that each subject is one row and each month is a column.
df_list <- df_summary %>%
  pivot_wider(id_cols = USUBJID,
              names_from = month,
              values_from = combined)

# The resulting data frames:
#   • df_egfr  - one row per USUBJID with one column per month (EGFRCKD values)
#   • df_hba1c - one row per USUBJID with one column per month (HBA1C values)
#   • df_bmi   - one row per USUBJID with one column per month (BMI values)
#   • df_list  - one row per USUBJID with one column per month; each cell is a list:
#                [EGFRCKD, HBA1C, BMI]


In [17]:
head(df_bmi)

USUBJID,0,12,24,36,48,6,60
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EX2211-3748/10012,31.6,31.6,31.6,31.6,31.6,31.6,31.6
EX2211-3748/10023,39.0,37.3,37.3,38.2,39.5,38.5,39.9
EX2211-3748/10028,30.8,30.7,30.4,29.3,29.3,30.6,30.1
EX2211-3748/10034,41.4,42.6,41.7,30.4,30.4,41.9,29.5
EX2211-3748/10050,50.8,48.8,45.8,45.2,45.2,45.1,44.7
EX2211-3748/10068,26.9,27.6,28.2,28.3,28.3,27.3,28.2


In [18]:
df_hba1c

USUBJID,0,12,18,24,3,30,36,42,48,54,6,60,1,61
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EX2211-3748/10012,7.1,7.1,7.1,7.1,7.1,7.1,7.1,7.1,7.1,7.1,7.1,7.1,NA,NA
EX2211-3748/10023,7.6,6.0,6.3,6.0,6.3,6.1,6.4,6.8,6.5,6.5,6.3,7.3,NA,NA
EX2211-3748/10028,9.9,9.5,7.9,9.6,8.4,9.2,9.9,9.6,9.6,9.6,8.6,9.6,NA,NA
EX2211-3748/10034,9.6,9.6,9.5,9.3,9.6,7.0,6.0,6.0,6.0,6.0,10.9,6.3,NA,NA
EX2211-3748/10050,7.5,7.8,8.9,8.7,8.0,8.5,8.1,8.1,8.1,8.1,6.3,9.6,NA,NA
EX2211-3748/10068,10.9,10.3,8.7,8.4,7.7,8.7,7.9,7.6,7.6,7.6,8.3,10.0,NA,NA
EX2211-3748/10085,7.6,5.9,6.1,6.0,7.6,5.8,6.1,5.8,5.8,5.8,6.1,5.8,NA,NA
EX2211-3748/10087,10.4,8.4,8.6,8.2,10.5,7.6,8.5,8.5,8.5,8.5,9.3,10.3,NA,NA
EX2211-3748/10090,10.0,6.7,6.7,7.5,7.3,7.0,7.8,7.2,8.3,8.3,6.3,7.8,NA,NA
